In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value


class IntegerValue:
    """
    This has memory leak - strong reference to the instance inside dict
    """
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance)


In [2]:
import weakref


class IntegerValue:
    """
    Improved verstion with a WeakKeyDictionary - has no memory leak, but still not ideal.
    Instance has to be hashable for example.
    """
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance)

In [3]:
class Point:
    x = IntegerValue()


p = Point()
print(hex(id(p)))

0x10a11fec0


In [4]:
p.x = 2.25
print(p.x)

2


In [5]:
print(list(Point.x.values.items()))
print(Point.x.values.keyrefs())

[(<__main__.Point object at 0x10a11fec0>, 2)]
[<weakref at 0x10a12b740; to 'Point' at 0x10a11fec0>]


In [6]:
del p

In [7]:
# after instance deletin WeakRefDictionary removes entry from the dict automatically
print(list(Point.x.values.items()))
print(Point.x.values.keyrefs())

[]
[]


In [8]:
class IntegerValue:
    """
    Different approach with regular dict - stored instance id as a key.
    Doesn't create strong reference, but what if id gets reused with other instance? That's possible
    """
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(id(instance))

In [9]:
class Point:
    x = IntegerValue()

    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        # makes the class non hashable
        return isinstance(other, Point) and self.x == other.x


p = Point(2.25)
print(hex(id(p)))

0x10a13d820


In [10]:
p.x = 11.43
print(p.x)

11


In [11]:
id(p), Point.x.values

(4464039968, {4464039968: 11})

In [12]:
p_id = id(p)
del p

In [13]:
ref_count(p_id)  # random stuff since reference count is 0 at this point and instance got removed

2

In [14]:
Point.x.values  # still inside the dict, even if object is dead

{4464039968: 11}

In [15]:
# it's possible to pass additonal callback functon when weakref is created
# this hook will be fired when weakref becomes dead
def obj_destroyed(obj):
    print(f"{obj} is being destroyed")

p = Point(10.25)
w = weakref.ref(p, obj_destroyed)

In [16]:
del p 

<weakref at 0x10a158c70; dead> is being destroyed


In [17]:
class IntegerValue:
    """
    Different approach with regular dict - stored instance id as a key.
    Doesn't create strong reference, but what if id gets reused with other instance? That's possible
    """
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self.remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        value_tuple = self.values.get(id(instance)) or (None, None)
        return value_tuple[1]

    def remove_object(self, weak_ref_obj):
        print(f"called remove_object with: {self}, {weak_ref_obj}")



In [18]:
class Point:
    x = IntegerValue()


In [19]:
p1 = Point()
p2 = Point()
p1.x = 22.2
p2.x = 33.3


In [20]:
print(Point.x.values)

{4464166768: (<weakref at 0x10a159d00; to 'Point' at 0x10a15c770>, 22), 4464166720: (<weakref at 0x10a15a570; to 'Point' at 0x10a15c740>, 33)}


In [21]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [22]:
del p1

called remove_object with: <__main__.IntegerValue object at 0x10a15c290>, <weakref at 0x10a159d00; dead>


In [23]:
class IntegerValue:
    """
    Different approach with regular dict - stored instance id as a key.
    Doesn't create strong reference, but what if id gets reused with other instance? That's possible
    """
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self.remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        value_tuple = self.values.get(id(instance)) or (None, None)
        return value_tuple[1]

    def remove_object(self, weak_ref_obj):
        reverse_lookup = [key for key, value in self.values.items() if value[0] is weak_ref_obj]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.values[key]
            print(f"Removed key: {key}")


class Point:
    x = IntegerValue()



In [24]:
p = Point()
p.x = 22.2

In [25]:
Point.x.values

{4464170656: (<weakref at 0x10a16c7c0; to 'Point' at 0x10a15d6a0>, 22)}

In [26]:
del p

Removed key: 4464170656


In [27]:
Point.x.values

{}

In [28]:
# when __slots__ are used, the instance __weakref__ is removed, we need to pass it manually so weak references can be created
class Person:
    __slots__ = "name", "__weakref__"

In [29]:
p = Person()
w = weakref.ref(p)

In [30]:
p, w

(<__main__.Person at 0x10a121750>,
 <weakref at 0x10a16e430; to 'Person' at 0x10a121750>)

In [31]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError("Value must be a string")

        if len(value) < self._min_length:
            raise ValueError(f"Value should be at least {self._min_length} characters")

        if len(value) > self._max_length:
            raise ValueError(f"Value should be no more than {self._max_length} characters")

        self.data[id(instance)] = (weakref.ref(instance, self._clean_data), value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self

        values_tuple = self.data.get(id(instance)) or (None, None)
        return values_tuple[1]

    def _clean_data(self, weak_ref_obj):
        print(f"_clean_data called with: {self}, {weak_ref_obj}")
        instance_id = [key for key, value in self.data.items() if value[0] is weak_ref_obj]
        if instance_id:
            del self.data[instance_id[0]]



In [32]:
class Person:
    __slots__ = "__weakref__",

    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)

    def __eq__(self, other):
        return isinstance(other, Person) and self.first == other.first_name and self.last_name == other.last_name

    def __repr__(self):
        return f"<{type(self).__name__} first_name={self.first_name} last_name={self.last_name}> @ {hex(id(self))}>"


class BankAccount:
    __slots__ = "__weakref__",

    account_number = ValidString(5, 255)
    
    def __repr__(self):
        return f"<{type(self).__name__} account_number={self.account_number}> @ {hex(id(self))}>"

    def __eq__(self, other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

In [33]:
p1 = Person()

try:
    p1.first_name = ""
except ValueError as e:
    print(e)

Value should be at least 1 characters


In [34]:
p2 = Person()
p1.first_name, p1.last_name = "Bob", "Smith"
p2.first_name, p2.last_name = "John", "Doe"

called remove_object with: <__main__.IntegerValue object at 0x10a15c290>, <weakref at 0x10a15a570; dead>


In [35]:
print(p1)
print(p2)

<Person first_name=Bob last_name=Smith> @ 0x10a15e450>
<Person first_name=John last_name=Doe> @ 0x10a180920>


In [36]:
b1, b2 = BankAccount(), BankAccount()

In [37]:
b1.account_number, b2.account_number = "Savings", "Other"

In [38]:
print(b1)
print(b2)

<BankAccount account_number=Savings> @ 0x10a181b50>
<BankAccount account_number=Other> @ 0x10a181d00>


In [39]:
print(Person.first_name.data)
print(Person.last_name.data)

{4464174160: (<weakref at 0x10a140d60; to 'Person' at 0x10a15e450>, 'Bob'), 4464314656: (<weakref at 0x10a1989a0; to 'Person' at 0x10a180920>, 'John')}
{4464174160: (<weakref at 0x10a198810; to 'Person' at 0x10a15e450>, 'Smith'), 4464314656: (<weakref at 0x10a198950; to 'Person' at 0x10a180920>, 'Doe')}


In [40]:
del p1

_clean_data called with: <__main__.ValidString object at 0x10a1813d0>, <weakref at 0x10a198810; dead>
_clean_data called with: <__main__.ValidString object at 0x10a180b90>, <weakref at 0x10a140d60; dead>


In [41]:
print(Person.first_name.data)
print(Person.last_name.data)

{4464314656: (<weakref at 0x10a1989a0; to 'Person' at 0x10a180920>, 'John')}
{4464314656: (<weakref at 0x10a198950; to 'Person' at 0x10a180920>, 'Doe')}
